In [0]:
from __future__ import absolute_import, division, print_function
import numpy as np
import keras
from keras.datasets import cifar10, mnist
from keras.models import Sequential# sequential is one type of model; there are graph models as well
from keras.layers import Dense, Activation, Dropout, Flatten, Reshape # Dense is fully connected layer
from keras.layers import Convolution2D, MaxPooling2D #
from keras.utils import np_utils
import pickle # serialization fancy word for storing on disk
from matplotlib import pyplot as plt
import seaborn as sns
plt.rcParams['figure.figsize'] = (15, 8)

In [18]:
from google.colab import drive
drive.mount('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [19]:
!ls "/drive/My Drive/Colab Notebooks/train_dataset"

 Black-grass	    'Common wheat'	'Scentless Mayweed'
 Charlock	    'Fat Hen'		'Shepherds Purse'
 Cleavers	    'Loose Silky-bent'	'Small-flowered Cranesbill'
'Common Chickweed'   Maize		'Sugar beet'


In [0]:
#PATH_OF_DATA= '/content/gdrive/"My Drive"/name_of_dataset_folder'
pathin_GDrvie = '/drive/My Drive/Colab Notebooks/train_dataset'
#!ls (pathin_GDrvie)



In [0]:
import os
os.chdir('/drive/My Drive/Colab Notebooks/train_dataset')

In [22]:
os.listdir()

['Maize',
 'Loose Silky-bent',
 'Small-flowered Cranesbill',
 'Shepherds Purse',
 'Common Chickweed',
 'Sugar beet',
 'Scentless Mayweed',
 'Cleavers',
 'Fat Hen',
 'Common wheat',
 'Charlock',
 'Black-grass']

In [23]:
x_train = []
y_train = []
import cv2
for i in os.listdir():
    print(i)
    if (os.path.isdir(i)):
            for j in os.listdir(i):
                try:
                    dummy = cv2.imread('/drive/My Drive/Colab Notebooks/train_dataset/' + i + "/" + j)
                    dummy = cv2.resize(dummy,(128,128))
                    x_train.append(dummy)
                    y_train.append(i)
                except Exception as e:
                    print(e)


Maize
Loose Silky-bent
Small-flowered Cranesbill
Shepherds Purse
Common Chickweed
Sugar beet
Scentless Mayweed
Cleavers
Fat Hen
Common wheat
Charlock
Black-grass


In [24]:
x_train[0].shape

(128, 128, 3)

In [0]:
import pandas as pd
dum = pd.get_dummies(y_train)

In [0]:
encoded_labels = dum
y_train = dum

In [0]:
import numpy as np
y_train = np.array(y_train)

In [0]:
x_train = np.array(x_train)

In [29]:
x_train[0].shape

(128, 128, 3)

In [30]:
from sklearn.model_selection import train_test_split
x_trainnew, x_val, y_trainnew, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=2)
print (len(x_trainnew))
print (len(x_val))

3800
950


In [32]:
x_trainnew[0].shape

(128, 128, 3)

In [0]:
x_train2 = x_trainnew.reshape(x_trainnew.shape[0],128,128,3)

In [0]:
x_val = x_val.reshape(x_val.shape[0],128,128,3)

In [35]:
print (x_trainnew.shape)
print (x_val.shape)

(3800, 128, 128, 3)
(950, 128, 128, 3)


In [36]:
print(y_trainnew.shape)
print(y_val.shape)

(3800, 12)
(950, 12)


In [0]:
from keras.models import Sequential
from keras.layers import Convolution2D, Dropout, Dense

In [0]:
from keras.layers import BatchNormalization
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.optimizers import adam

In [39]:
model = Sequential()
model.add(BatchNormalization(input_shape = (128,128,3)))
model.add(Convolution2D(32, (3,3), activation ='relu', input_shape = (128, 128, 3))) 
model.add(MaxPooling2D(pool_size=2))
#model.add(Dropout(0.8))

model.add(Convolution2D(filters=64, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
#model.add(Dropout(0.8))

model.add(Convolution2D(filters=128, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
#model.add(Dropout(0.8))

model.add(Convolution2D(filters=128, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
#model.add(Dropout(0.8))

model.add(Flatten()) 

# fully connected layer
model.add(Dense(units=128,activation = 'relu'))
model.add(Dense(units = 64, activation = 'relu'))
#model.add(Dropout(0.3))
model.add(Dense(units = 32, activation = 'relu'))
#model.add(Dropout(0.3))
model.add(Dense(units = 12, activation = 'softmax')) 


Instructions for updating:
Colocations handled automatically by placer.


In [0]:
optimizer = adam(lr=0.001)
model.compile(optimizer='adam', loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [41]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 128, 128, 3)       12        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 126, 126, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 63, 63, 64)        32832     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 31, 31, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 31, 31, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 15, 15, 128)       0         
__________

In [44]:
print(y_trainnew.shape)

(3800, 12)


In [43]:
batch_size = 64
nb_epochs = 10
history = model.fit(x_trainnew,y_trainnew,
                    epochs=20, 
                    validation_data=(x_val,y_val),
                    verbose = 1,
                    initial_epoch=0)

Instructions for updating:
Use tf.cast instead.
Train on 3800 samples, validate on 950 samples
Epoch 1/20
3800/3800 [==============================] - 10s 3ms/step - loss: 2.0814 - acc: 0.2700 - val_loss: 1.6408 - val_acc: 0.3937
Epoch 2/20
3800/3800 [==============================] - 8s 2ms/step - loss: 1.3007 - acc: 0.5458 - val_loss: 1.0281 - val_acc: 0.6611
Epoch 3/20
3800/3800 [==============================] - 8s 2ms/step - loss: 0.8920 - acc: 0.7003 - val_loss: 0.8297 - val_acc: 0.7253
Epoch 4/20
3800/3800 [==============================] - 8s 2ms/step - loss: 0.6510 - acc: 0.7892 - val_loss: 0.7145 - val_acc: 0.7653
Epoch 5/20
3800/3800 [==============================] - 8s 2ms/step - loss: 0.5256 - acc: 0.8176 - val_loss: 0.7600 - val_acc: 0.7453
Epoch 6/20
3800/3800 [==============================] - 8s 2ms/step - loss: 0.4220 - acc: 0.8513 - val_loss: 0.6456 - val_acc: 0.8000
Epoch 7/20
3800/3800 [==============================] - 8s 2ms/step - loss: 0.3524 - acc: 0.8763 - v